![logo](Data/logo.png)

This notebook shows how to load data from a netcdf using `xarray` file and plot them in a coordinate system using cartopy.

In [1]:
#import library
import xarray as xr
import netCDF4
import cartopy
import cftime
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pyproj

We use here a netcdf file that include output from a MesoNH simulation.

In [2]:
fileinput = '/data/IMFSE/PythonCourse/CDS/CERRA/cerra-2000.nc'

to open the netcdf file for this file, we need to set `decode_times` to False, as the time units in the file was not set properly.

In [3]:
ds = xr.open_dataset(fileinput)

In [4]:
ds

<xarray.Dataset> Size: 27GB
Dimensions:     (valid_time: 1464, y: 1069, x: 1069)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 12kB 2000-01-01 ... 2000-12-31T18...
    latitude    (y, x) float64 9MB ...
    longitude   (y, x) float64 9MB ...
    expver      (valid_time) <U4 23kB ...
Dimensions without coordinates: y, x
Data variables:
    wdir10      (valid_time, y, x) float32 7GB ...
    si10        (valid_time, y, x) float32 7GB ...
    r2          (valid_time, y, x) float32 7GB ...
    t2m         (valid_time, y, x) float32 7GB ...
Attributes:
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Norrkoping
    history:                 2024-08-08T16:29 GRIB to CDM+CF via cfgrib-0.9.1...

you can see the time unit using netCDF4 which is a less lower level library that deals directly with netCDF file structures. It requires more manual handling of the data, such as keeping track of dimensions and coordinates separately. 

In [5]:
nc = netCDF4.Dataset(fileinput)
nc['time']

IndexError: time not found in /

units should read `seconds since: 2022-07-16 00:00:00.000000` to be correctly interpreted in `xarray`  
To correct this we run the following commands:

In [ ]:
# Extract the time variable
time_var = ds["time"]

# Get the time units, and remove the non-standard prefix
units = time_var.attrs['units'].replace('fire ignition: ', '')

# Convert the time variable using cftime
times = cftime.num2date(time_var.values, units=units, calendar='standard')

# Convert to pandas datetime if needed
times_as_datetime = [datetime(year=t.year, month=t.month, day=t.day, 
                              hour=t.hour, minute=t.minute, second=t.second,microsecond=t.microsecond)
                     for t in times]


# Replace the time variable in the dataset with the converted times
ds["time"] = ("time", times_as_datetime)

# Print the dataset to check the time conversion
print(ds["time"])

the dataset is now well define. below is an overview of the data

In [ ]:
ds

In [ ]:
#now for example to print the topography you can use
ds['topography'].plot()

In [ ]:
#or to plot the time series of the temperature in the center of the domain
ds['Temp'][:,0,75,75].plot()

In the plot above you can see the diurnal cycle of the temperature that picks around 12pm and get to its lowest at 6am.

lets now plot the terrain data on a geographic coordinate system. The plot above is showing the data on the cartesian grid of the domain simulation.

In [ ]:
# Plot the data

crs_here = ccrs.PlateCarree() 

fig, ax = plt.subplots(subplot_kw={'projection': crs_here})

# Add borders, coastlines and gridlines
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
gl = ax.gridlines(draw_labels=True)

# Disable labels on the right and top sides
gl.right_labels = False
gl.top_labels = False

# Plot data using pcolormesh
colm = ax.pcolormesh(ds.lon, ds.lat, ds.topography, cmap='viridis', shading='auto', transform=crs_here)

# Add a colorbar
plt.colorbar(colm, ax=ax, orientation='vertical', label='elevation (m)')

# Set title
plt.title('topography')

plt.show()

lest now project this map on a UTM projection coordinate system [epsg:25831](https://epsg.io/25831).

In [ ]:
# Define the source and target CRS
source_crs = "EPSG:4326"  # WGS84 - Latitude/Longitude
target_crs = "EPSG:25831" # ETRS89 / UTM zone 31N

# Create a Transformer object
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Transform the coordinates
easting, northing = transformer.transform(ds.lon, ds.lat)

In [ ]:
# Plot the data

crs_here = ccrs.epsg(25831) 

fig, ax = plt.subplots(subplot_kw={'projection': crs_here})

# Add borders, coastlines and gridlines
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
gl = ax.gridlines(draw_labels=True)

# Disable labels on the right and top sides
gl.right_labels = False
gl.top_labels = False

# Plot data using pcolormesh
colm = ax.pcolormesh(easting, northing, ds.topography, cmap='viridis', shading='auto', transform=crs_here)

# Add a colorbar
plt.colorbar(colm, ax=ax, orientation='vertical', label='elevation (m)')

# Set title
plt.title('topography')

plt.show()

See that now we have here again an aspect ration of 1 (the image is a square). In the Platecarree projection above, the aspect ration is different from 1 as 1 degree of longitude is only equivalent in distance to 1 degree of latitude at the equator. As you can see below meridian are getting closer at higher/lower latitude in the north/south pole.

In [ ]:
plt.figure(figsize=(3, 3))
ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines(resolution='110m')
ax.gridlines()